# PASCAL VOC 

In [1]:
from torchvision import datasets
import torch
import numpy as np
from torch.utils.data import ConcatDataset
import clip
import os

In [2]:
trainval_dataset_2012 = datasets.VOCDetection(root="/home/ubuntu/mmdetection_od/mmdetection/data", year="2012", image_set="trainval", download=False)

In [3]:
len(trainval_dataset_2012)

11540

In [4]:
trainval_dataset_2007 = datasets.VOCDetection(root="/home/ubuntu/mmdetection_od/mmdetection/data",year="2007",image_set="trainval", download=False)

In [5]:
len(trainval_dataset_2007)

5011

In [6]:
train_dataset = ConcatDataset([trainval_dataset_2007, trainval_dataset_2012])

In [7]:
len(train_dataset)

16551

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# to use a different clip model
model, preprocess = clip.load("RN50", device=device)
#model, preprocess = clip.load('ViT-L/14', device=device)

In [ ]:
model

# Train embeddings

In [13]:
image_features = []
image_file_names=[]

# iterate over the dataset 
for idx in range(len(train_dataset)):
    # get the image and its annotations
    img, target = train_dataset[idx]
    image =  preprocess(img).unsqueeze(0).to(device)
    
    with torch.no_grad():
        image_features.append(torch.squeeze(model.encode_image(image)))
        image_file_names.append(target['annotation']['filename'])

100%|██████████| 16551/16551 [20:18<00:00, 13.59it/s]


image_features

In [14]:
image_features[0].shape

torch.Size([1024])

In [15]:
len(image_features)

16551

In [17]:
image_features_cpu=[image_feature.cpu().numpy() for image_feature in image_features]

# Save features

In [19]:
#Save features in the correct results folder
dataset='pascalvoc'
results_folder = os.path.join('/home/ubuntu/master_thesis/covering_lens/TypiClust/scan','results')
if not os.path.exists(results_folder):
        os.makedirs(results_folder)
results_folder_dataset = os.path.join(results_folder,dataset)
if not os.path.exists(results_folder_dataset):
        os.makedirs(results_folder_dataset)
results_folder_dataset_pretext = os.path.join(results_folder_dataset,'pretext')
if not os.path.exists(results_folder_dataset_pretext):
        os.makedirs(results_folder_dataset_pretext)
np.save(os.path.join(results_folder_dataset_pretext,'features_seed1_clip'), image_features_cpu)

In [20]:
check_emb=np.load('/home/ubuntu/master_thesis/covering_lens/TypiClust/scan/results/pascalvoc/pretext/features_seed1_clip.npy')

In [22]:
check_emb.shape

(16551, 1024)

In [24]:
len(image_file_names)

16551

In [25]:
with open(os.path.join(results_folder_dataset_pretext, 'filenames.txt'), 'w') as f:
    for image_file_name in image_file_names:
        f.write(image_file_name + '\n')

: 